In [1]:
import hidet

In [2]:
def conv_relu(x, out_channels, kernel, stride, padding):
    w = hidet.randn(shape=[out_channels, x.shape[1], kernel, kernel])
    b = hidet.randn(shape=[1, out_channels, 1, 1])
    x = hidet.ops.conv_pad(x, pads=padding)
    x = hidet.ops.conv2d(x, w, stride=stride) + b
    x = hidet.ops.relu(x)
    return x

img = hidet.symbol([1, 3, 224, 224])
out = conv_relu(img, 64, 7, 2, 3)
graph = hidet.trace_from(out, inputs=[img])
print(graph)


Graph(x: float32[1, 3, 224, 224]){
  c = Constant(float32[64, 3, 7, 7])
  c_1 = Constant(float32[1, 64, 1, 1])
  x_1: float32[1, 3, 230, 230] = Pad(x, pads=[0, 0, 3, 3, 0, 0, 3, 3], mode="constant", value=0.0)  
  x_2: float32[1, 64, 112, 112] = Conv2d(x_1, c, stride=[2, 2], groups=1, dilations=(1, 1))  
  x_3: float32[1, 64, 112, 112] = Add(x_2, c_1)  
  x_4: float32[1, 64, 112, 112] = Relu(x_3)  
  return x_4
}
